In [1]:
import sys
import os

current_dir = os.getcwd()
src_dir = os.path.join(current_dir, '..', 'src')
sys.path.append(src_dir)

import pandas as pd
from dqc.dqc import DataQualityPipeline
from etl.etl import ETL, RelationsTransform
from etl.kaggle_data_loader import download_data

In [2]:
download_data()

data_paths = {}

for dirname, _, filenames in os.walk(f'{src_dir}/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        data_paths[filename] = f"{dirname}/{filename}"

relations = {}

for file, path in data_paths.items():
    relations[file.split('.')[0]] = pd.read_csv(path)

/home/sandra/Projects/ds_practice/notebooks/../src/data/sales_train.csv
/home/sandra/Projects/ds_practice/notebooks/../src/data/item_categories.csv
/home/sandra/Projects/ds_practice/notebooks/../src/data/sample_submission.csv
/home/sandra/Projects/ds_practice/notebooks/../src/data/items.csv
/home/sandra/Projects/ds_practice/notebooks/../src/data/test.csv
/home/sandra/Projects/ds_practice/notebooks/../src/data/shops.csv


## I. Data quality check:
1. Identify the data sources and gather the data
2. Check for missing values, duplicates, and outliers
3. Check for inconsistencies in data types and formats
4. Validate the data against business rules and requirements
5. Identify any data quality issues and document them
6. Determine whether to clean, transform, or discard the data

In [3]:
report = DataQualityPipeline()(relations)

In [4]:
report["validation report"]['values']

nan_report  unique    dtype    total
table_name        column_name                                            
item_categories   item_category_name        0.0      84   object       84
                  item_category_id          0.0      84    int64       84
items             item_category_id          0.0      84    int64    22170
                  item_id                   0.0   22170    int64    22170
                  item_name                 0.0   22170   object    22170
sales_train       date                      0.0    1034   object  2935849
                  date_block_num            0.0      34    int64  2935849
                  shop_id                   0.0      60    int64  2935849
                  item_id                   0.0   21807    int64  2935849
                  item_price                0.0   19993  float64  2935849
                  item_cnt_day              0.0     198  float64  2935849
sample_submission ID                        0.0  214200    int64   214200
                  item_cnt_month            0.0       1  float64   214200
shops             shop_name                 0.0      60   object       60
                  shop_id                   0.0      60    int64       60
test              ID                        0.0  214200    int64   214200
                  shop_id                   0.0      42    int64   214200
                  item_id                   0.0    5100    int64   214200

* item_cnt_month not informative, 1 unique value
* item_category_id and shop_id have average cardinality (84, 60)
* item_id have high cardinality (22170) and should be processed

In [5]:
report["validation report"]['duplicates']

,duplicates
item_categories,0
items,0
sales_train,6
sample_submission,0
shops,0
test,0


In [6]:
report["validation report"]['sparcity']

,left_only,intersect,right_only
"sales_train, items ON item_id",0,21807,363
"sales_train, test ON item_id",17070,4737,363
"sales_train, test ON shop_id",18,42,0
"sales_train, shops ON shop_id",0,60,0
"item_categories, items ON item_category_id",0,84,0
"sample_submission, test ON ID",0,214200,0
"items, test ON item_id",17070,5100,0
"test, shops ON shop_id",0,42,18


* sales_train and test sets contain 
item_ids which not represented in each other => for some item_ids we don't know price

* on the contrary, shop_ids in test all represented in sales_train

In [7]:
report["statistical report"]['distribution']

,count,mean,std,min,1%,25%,50%,75%,99%,max
item_price,2935849.0,890.853233,1729.799631,-1.0,5.0,249.0,399.0,999.0,5999.0,307980.0
item_cnt_day,2935849.0,1.242641,2.618834,-22.0,1.0,1.0,1.0,1.0,5.0,2169.0
item_cnt_month,214200.0,0.500000,0.000000,0.5,0.5,0.5,0.5,0.5,0.5,0.5


* item_price and item_cnt_day contain negative values and outliers

In [8]:
report["statistical report"]['outliers']

,1% left range,98% middle range,1% right range
item_price,6.0,5994.0,301981.0
item_cnt_day,23.0,4.0,2164.0
item_cnt_month,0.0,0.0,0.0


* oultiers in item_price and item_cnt_day
* as was mentioned, item_cnt_month not informative and can be dropped

## ETL:
1. Extract the data from the source systems
2. Transform the data to meet the target schema and requirements
3. Load the transformed data into the target system
4. Validate the data after loading to ensure accuracy and completeness
5. Monitor the ETL process for errors and performance issues

In [9]:
# TO DO: add categorical data filters
filter_pipes = {
        "sales_train": [lambda df: df[(-10 < df["item_price"]) & (df["item_price"]  <= 60000)],
                        lambda df: df[(0 <= df["item_cnt_day"]) & (df["item_cnt_day"] <= 4.0)],
                        lambda df: df[~df.duplicated(keep="first")],
                        ],
    }

transform = RelationsTransform(filter_pipes)

etl = ETL(transform)
full_relations = etl.load()
full_relations

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
2,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
3,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""
4,10.01.2013,0,25,2564,349.00,1.0,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Музыка - Музыкальное видео,"Москва ТРК ""Атриум"""
...,...,...,...,...,...,...,...,...,...,...
2890596,10.10.2015,33,25,7409,299.00,1.0,V/A Nu Jazz Selection (digipack),55,Музыка - CD локального производства,"Москва ТРК ""Атриум"""
2890597,09.10.2015,33,25,7460,299.00,1.0,V/A The Golden Jazz Collection 1 2CD,55,Музыка - CD локального производства,"Москва ТРК ""Атриум"""
2890598,14.10.2015,33,25,7459,349.00,1.0,V/A The Best Of The 3 Tenors,55,Музыка - CD локального производства,"Москва ТРК ""Атриум"""
2890599,22.10.2015,33,25,7440,299.00,1.0,V/A Relax Collection Planet MP3 (mp3-CD) (jewel),57,Музыка - MP3,"Москва ТРК ""Атриум"""
